<a href="https://colab.research.google.com/github/s-mostafa-a/lung-cancer-segmentation/blob/master/using_PartiallyReversibleUnet_for_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def _cn_t_d():
  from google.colab import drive
  drive.mount("/content/drive")

In [2]:
def _untar_data():
  !tar -xf ./drive/My\ Drive/Copy_Task06_Lung.tar -C ./

In [3]:
def connect_to_drive():
  !mkdir drive
  _cn_t_d()
  _untar_data()

In [4]:
def install_all_needed_packages():
  !git clone https://github.com/s-mostafa-a/PartiallyReversibleUnet
  !pip install git+git://github.com/jcreinhold/niftidataset.git
  !pip install comet-ml
  !pip install MedPy
  !pip install h5py
  !pip install nibabel
  !pip install scikit-image
  !pip install revtorch

# opencv-python
# matplotlib
# h5py
# -e git+https://github.com/lmkoch/medpy/#egg=medpy
# revtorch
# scikit-image
# nibabel
# comet-ml

In [5]:
connect_to_drive()
install_all_needed_packages()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Cloning into 'PartiallyReversibleUnet'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 67 (delta 2), reused 8 (delta 2), pack-reused 56
Unpacking objects: 100% (67/67), done.
  Cloning git://github.com/jcreinhold/niftidataset.git to /tmp/pip-req-build-hynd06tj
  Running command git clone -q git://github.com/jcreinhold/niftidataset.git /tmp/pip-req-build-hynd06t

Because 3d image size are in like this: (512,512,z) which z range is from 112 to 465, I thougth it would be good if I crop each 3D image from it's third axis (z) and fix that in a constant like 110. This way raw data loss becomes so little. for example if we have an image sized (512,512, 112) we would generate an image (512,512,110), Also if we had an image sized (512, 512, 373) we would generate 3 images with sizes (512,512,110) which it covers 330 out of 373 from z of original raw data.

But when I used it with size (512,512,110), cuda went out of memory and it needed more RAM. Also I found some paper which they cropped this dataset's images to size (64,64,64)! which I thought their data loss would be so high. But anyway I went a way similar to their way and cropped original images to the size (110,110,110) which almost contains 8 times more data (and less data loss) from that paper.

So I wrote below classes which use some features of [this code](https://github.com/jcreinhold/niftidataset). It used a single image 5 times and cropped each randomly, but I thought It will not be useful because np.random.seed() makes each time just one crop. officially it wouldn't help.


In [6]:
!rm -r PartiallyReversibleUnet
install_all_needed_packages()

Cloning into 'PartiallyReversibleUnet'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 67 (delta 2), reused 8 (delta 2), pack-reused 56
Unpacking objects: 100% (67/67), done.
  Cloning git://github.com/jcreinhold/niftidataset.git to /tmp/pip-req-build-h1nmgt72
  Running command git clone -q git://github.com/jcreinhold/niftidataset.git /tmp/pip-req-build-h1nmgt72
  Created wheel for niftidataset: filename=niftidataset-0.2.0-cp36-none-any.whl size=12356 sha256=7a7c8a67672e6dc9c7a8e473dc8b2243e0d4f511307dc11e5ece9e06149eff03
  Stored in directory: /tmp/pip-ephem-wheel-cache-tdzj4sl7/wheels/3d/60/12/b5ae6069c6d1cf966d06eb7d8336abd8f826601c3f31f17c3e
Successfully built niftidataset


In [7]:
import numpy as np
import os
import sys
module_path = os.path.abspath(os.path.join('/content/PartiallyReversibleUnet'))
if module_path not in sys.path:
    sys.path.append(module_path)
print(sys.path)

['', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython', '/content/PartiallyReversibleUnet']


In [8]:
from niftidataset import NiftiDataset, train_val_split, RandomCrop3D, ToTensor, Normalize, TrimIntensity
from torchvision import transforms as tvt
from torch.nn import BCEWithLogitsLoss
import torch 
from fastai import vision as faiv
%matplotlib inline
%load_ext autoreload
%autoreload 2
!export CUDA_LAUNCH_BLOCKING=1

image size must be dividable to 32 because it halves 5 times.

In [ ]:
from train import main
np.random.seed(1)
train_dir = '/content/Task06_Lung/'
tfms = tvt.Compose([ToTensor(), RandomCrop3D(96), TrimIntensity(-1000.0,1000.0,0.0,255.0), Normalize(is_3d=True)])
tds, vds = train_val_split(source_dir=train_dir+'imagesTr', target_dir=train_dir+'labelsTr', valid_pct=0.2, transform=tfms)
main(tds, vds)

Not logging to comet.ml
Parameters: 12'360'423
using cpu
======= RUNNING EXPERIMENT =======
Reversible NO_NEW60, dropout
ID: 8990740451391834330
[0,     5] loss: 0.200
[0,    10] loss: 0.200
